IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\FAULT_M3_10


VOLO m3 FAULT 10%

In [3]:
rcou_m3_fault10 = pd.read_csv("RCOU.csv")
rcou_m3_fault10 = rcou_m3_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

rcou_m3_fault10 = rcou_m3_fault10[((rcou_m3_fault10['C9'] >= 1400) & (rcou_m3_fault10['C10'] >= 1400) & (rcou_m3_fault10['C11'] >= 1400) & (rcou_m3_fault10['C12'] >= 1400) & (rcou_m3_fault10['C13'] >= 1400) & (rcou_m3_fault10['C14']>= 1400))]

pd.set_option('display.max_rows',700)
display(rcou_m3_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
518,106891806,1479,1406,1428,1459,1460,1427
519,106907540,1455,1450,1499,1401,1499,1402
523,106952242,1456,1497,1474,1479,1451,1501
526,106984710,1422,1557,1490,1497,1414,1563
527,106995645,1491,1507,1473,1524,1460,1535
...,...,...,...,...,...,...,...
7705,186260603,1491,1520,1547,1462,1459,1550
7706,186270800,1556,1446,1524,1482,1505,1502
7707,186281436,1556,1438,1522,1478,1504,1495
7708,186291822,1552,1431,1521,1467,1504,1485


In [4]:
xkf1_m3_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m3_fault10 = xkf1_m3_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m3_fault10 = xkf1_m3_fault10[((xkf1_m3_fault10['TimeUS'] >= 106891806) & (xkf1_m3_fault10['TimeUS'] <= 186302079))]
print(xkf1_m3_fault10)

         TimeUS  Roll  Pitch    Yaw
520   106905542 -0.33   0.35  12.46
521   106918323 -0.39   0.39  12.45
522   106928766 -0.20   0.37  12.46
523   106939536 -0.15   0.37  12.46
524   106950235 -0.17   0.36  12.45
...         ...   ...    ...    ...
7707  186257957 -0.87   1.00   0.92
7708  186268729 -0.89   1.01   0.95
7709  186278947 -0.97   1.00   0.99
7710  186289650 -1.03   0.97   1.00
7711  186299874 -1.02   0.92   1.03

[7192 rows x 4 columns]


In [5]:
att_m3_fault10 = pd.read_csv("ATT.csv")
att_m3_fault10 = att_m3_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
#att = att[(att['TimeUS'] - att['TimeUS'].iat[0])]
att_m3_fault10 = att_m3_fault10[((att_m3_fault10['TimeUS'] >= 106891806) & (att_m3_fault10['TimeUS'] <= 186302079))]
print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
520   106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
521   106919694     0.02 -0.39      0.00   0.39   12.39  12.45
522   106930531     0.00 -0.20      0.00   0.37   12.39  12.46
523   106941454     0.00 -0.15      0.00   0.37   12.39  12.46
524   106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7707  186259757     1.18 -0.87     -0.69   1.00    4.83   0.92
7708  186269991     1.25 -0.89     -0.68   1.01    4.83   0.95
7709  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7710  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7711  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03

[7192 rows x 7 columns]


In [6]:
att_m3_fault10 = pd.merge_asof(att_m3_fault10,xkf1_m3_fault10, on="TimeUS", by=["Roll","Pitch","Yaw"], tolerance=100000)
print(att_m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45
...         ...      ...   ...       ...    ...     ...    ...
7187  186259757     1.18 -0.87     -0.69   1.00    4.83   0.92
7188  186269991     1.25 -0.89     -0.68   1.01    4.83   0.95
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03

[7192 rows x 7 columns]


In [7]:
esc_0_m3_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m3_fault10 = esc_0_m3_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
#att = att[(att['TimeUS'] - att['TimeUS'].iat[0])]
esc_0_m3_fault10 = esc_0_m3_fault10[((esc_0_m3_fault10['TimeUS'] >= 106891806) & (esc_0_m3_fault10['TimeUS'] <= 186302079))]
print(esc_0_m3_fault10)

        TimeUS   RPM  Curr
48   106915076  4052  3.84
49   107021707  4527  3.33
50   107136109  5207  4.03
51   107243037  5567  4.16
52   107353705  6126  4.11
53   107470606  6292  4.03
54   107566485  6110  5.02
55   107679521  6329  4.91
56   107783468  6555  4.72
57   107894572  6519  4.78
58   108112498  6629  5.25
59   108217504  6698  4.79
60   108336499  6613  4.50
61   108444698  6470  4.41
62   108556638  6434  4.30
63   108671665  6215  4.45
64   108766005  6373  4.41
65   108876603  6243  4.08
66   108991457  6292  4.11
67   109100290  6367  3.95
68   109206496  6278  4.05
69   109316758  6115  4.06
70   109426847  6138  4.40
71   109538592  6089  4.36
72   109659419  6308  4.47
73   109754045  6276  4.52
74   109861091  6415  4.43
75   109969754  6438  4.20
76   110077487  6283  4.79
77   110179081  6439  4.60
78   110288200  6394  4.65
79   110397495  6640  4.53
80   110514215  6523  4.68
81   110622934  6474  4.54
82   110729483  6514  4.91
83   110835231  6518  4.56
8

In [8]:
imu_2_m3_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m3_fault10 = imu_2_m3_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
#att = att[(att['TimeUS'] - att['TimeUS'].iat[0])]
imu_2_m3_fault10 = imu_2_m3_fault10[((imu_2_m3_fault10['TimeUS'] >= 106891806) & (imu_2_m3_fault10['TimeUS'] <= 186302079))]
print(imu_2_m3_fault10)

          TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ
2061   106893398 -0.181538 -0.083858  0.015945  0.338709  0.538201  -9.890386
2062   106896186 -0.206260 -0.052126  0.009905  0.349124  0.591406  -9.960483
2063   106898723 -0.009462  0.044965 -0.007002  0.346877  0.739314  -9.861938
2064   106902129  0.214731  0.027732 -0.018386  0.314985  0.745321  -9.716557
2065   106904697  0.267047 -0.079066 -0.034859  0.257307  0.660825  -9.680092
...          ...       ...       ...       ...       ...       ...        ...
29394  186290860 -0.068042 -0.066280  0.023124  0.583846  1.092862 -10.782830
29395  186293804  0.018301 -0.103987  0.024436  0.617982  1.217526 -10.874960
29396  186296514  0.119775 -0.107886  0.029726  0.623442  1.229517 -10.930940
29397  186298682  0.088126 -0.105963  0.042768  0.641600  1.321502 -11.139120
29398  186301070  0.025372 -0.119452  0.060226  0.609978  1.467409 -11.271120

[27338 rows x 7 columns]


In [9]:
m3_fault10 = pd.merge_asof(att_m3_fault10, imu_2_m3_fault10, on="TimeUS",  tolerance= 100000)
print(m3_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw    Yaw      GyrX  \
0     106906700     0.04 -0.33     -0.01   0.35   12.39  12.46 -0.105342   
1     106919694     0.02 -0.39      0.00   0.39   12.39  12.45 -0.295167   
2     106930531     0.00 -0.20      0.00   0.37   12.39  12.46  0.116438   
3     106941454     0.00 -0.15      0.00   0.37   12.39  12.46  0.126498   
4     106951490    -0.02 -0.17      0.00   0.36   12.39  12.45 -0.012492   
...         ...      ...   ...       ...    ...     ...    ...       ...   
7187  186259757     1.18 -0.87     -0.69   1.00    4.83   0.92 -0.099492   
7188  186269991     1.25 -0.89     -0.68   1.01    4.83   0.95 -0.039038   
7189  186280650     1.31 -0.97     -0.67   1.00    4.83   0.99 -0.046706   
7190  186291048     1.36 -1.03     -0.64   0.97    4.83   1.00 -0.068042   
7191  186301226     1.39 -1.02     -0.59   0.92    4.83   1.03  0.025372   

          GyrY      GyrZ      AccX      AccY       AccZ  
0    -0.102818 -0.024055  0.1

In [10]:
m3_fault10 = pd.merge_asof(esc_0_m3_fault10, m3_fault10, on="TimeUS",  tolerance= 100000)
print(m3_fault10)

        TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw  \
0    106915076  4052  3.84     0.04  -0.33     -0.01   0.35   12.39  12.46   
1    107021707  4527  3.33    -0.01  -0.09      0.04   0.30   12.39  12.55   
2    107136109  5207  4.03     0.06   0.05      0.14  -0.10   12.39  12.92   
3    107243037  5567  4.16     0.01   0.37      0.16  -0.47   12.39  13.62   
4    107353705  6126  4.11    -0.21   0.60      0.27  -0.43   12.39  15.08   
5    107470606  6292  4.03    -0.74   0.71      0.68  -0.72   12.39  17.04   
6    107566485  6110  5.02    -1.49   1.06      1.07  -0.56   12.38  18.71   
7    107679521  6329  4.91    -2.60   1.26      1.54  -0.39   12.36  20.37   
8    107783468  6555  4.72    -3.50   0.51      2.15  -0.31   12.23  21.48   
9    107894572  6519  4.78    -4.30  -0.88      2.57   0.26   11.34  22.16   
10   108112498  6629  5.25    -5.37  -2.99      2.86   1.96    7.22  21.11   
11   108217504  6698  4.79    -5.52  -3.39      2.86   2.29    6

In [11]:
m3_fault10["TimeUS"] = m3_fault10["TimeUS"] - m3_fault10.iloc[0]["TimeUS"]
m3_fault10["TimeUS"] = m3_fault10["TimeUS"].astype(int)
m3_fault10["Guasto"] = 2
print(m3_fault10)

       TimeUS   RPM  Curr  DesRoll   Roll  DesPitch  Pitch  DesYaw    Yaw  \
0           0  4052  3.84     0.04  -0.33     -0.01   0.35   12.39  12.46   
1      106631  4527  3.33    -0.01  -0.09      0.04   0.30   12.39  12.55   
2      221033  5207  4.03     0.06   0.05      0.14  -0.10   12.39  12.92   
3      327961  5567  4.16     0.01   0.37      0.16  -0.47   12.39  13.62   
4      438629  6126  4.11    -0.21   0.60      0.27  -0.43   12.39  15.08   
5      555530  6292  4.03    -0.74   0.71      0.68  -0.72   12.39  17.04   
6      651409  6110  5.02    -1.49   1.06      1.07  -0.56   12.38  18.71   
7      764445  6329  4.91    -2.60   1.26      1.54  -0.39   12.36  20.37   
8      868392  6555  4.72    -3.50   0.51      2.15  -0.31   12.23  21.48   
9      979496  6519  4.78    -4.30  -0.88      2.57   0.26   11.34  22.16   
10    1197422  6629  5.25    -5.37  -2.99      2.86   1.96    7.22  21.11   
11    1302428  6698  4.79    -5.52  -3.39      2.86   2.29    6.45  19.48   

In [12]:
path_file = path_file.replace(r"m3\FAULT_m3_10", "")
os.chdir(path_file)
m3_fault10.to_csv('m3_fault10.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M3\FAULT_M3_10
